In [1]:
from distributed import LocalCluster, Client
cluster = LocalCluster()
cluster

LocalCluster(8d8c2f98, 'tcp://127.0.0.1:40579', workers=4, threads=12, memory=18.56 GiB)

In [2]:
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 12,Total memory: 18.56 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40579,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 18.56 GiB
Comm: tcp://127.0.0.1:40807,Total threads: 3
Dashboard: http://127.0.0.1:39365/status,Memory: 4.64 GiB
Nanny: tcp://127.0.0.1:46339,


In [3]:
from scipy.integrate import quad
import numpy as np
import uproot
from math import pi
from IPython.display import display

def get_Bx(x, y, z, x0, y0, z0, B0, R):
    def integrand_x(t, x, y, z):
        r_mag = np.sqrt((x-R*np.cos(t))**2 + (y-R*np.sin(t))**2 + z**2)
        r_3 = r_mag**3
        return R*z*np.cos(t)/r_3
    a, b = 0, 2*np.pi
    bx = B0*quad(integrand_x, a, b, args=(x-x0, y-y0, z-z0))[0]
    return bx

def get_By(x, y, z, x0, y0, z0, B0, R):
    def integrand_y(t, x, y, z):
        r_mag = np.sqrt((x-R*np.cos(t))**2 + (y-R*np.sin(t))**2 + z**2)
        r_3 = r_mag**3
        return R*z*np.sin(t)/r_3
    a, b = 0, 2*np.pi
    by = B0*quad(integrand_y, a, b, args=(x-x0, y-y0, z-z0))[0]
    return by

def get_Bz(x, y, z, x0, y0, z0, B0, R):
    def integrand_z(t, x, y, z):
        r_mag = np.sqrt((x-R*np.cos(t))**2 + (y-R*np.sin(t))**2 + z**2)
        r_3 = r_mag**3
        return R*(R - y*np.sin(t) - x*np.cos(t))/r_3
    a, b = 0, 2*np.pi
    bz = B0*quad(integrand_z, a, b, args=(x-x0, y-y0, z-z0))[0]
    return bz

def two_rings_x(x,y,z, x0,y0,z0, x1,y1,z1, B0, R):
    Bx_ring1 = get_Bx(x,y,z, x0,y0,z0, B0,R)
    Bx_ring2 = get_Bx(x,y,z, x1,y1,z1, B0,R)
    return Bx_ring1 + Bx_ring2

def two_rings_y(x,y,z, x0,y0,z0, x1,y1,z1, B0, R):
    By_ring1 = get_By(x,y,z, x0,y0,z0, B0,R)
    By_ring2 = get_By(x,y,z, x1,y1,z1, B0,R)
    return By_ring1 + By_ring2

def two_rings_z(x,y,z, x0,y0,z0, x1,y1,z1, B0, R):
    Bz_ring1 = get_Bz(x,y,z, x0,y0,z0, B0,R)
    Bz_ring2 = get_Bz(x,y,z, x1,y1,z1, B0,R)
    return Bz_ring1 + Bz_ring2


# Set parameters
l0 = 2*pi
B0 = 1
R = 0.25*l0

Lx           = 2.*l0
Ly           = 2.*l0
Lz           = 2.*l0

#Ring 1
x0 = Lx/2
y0 = Ly/2
z0 = 0

#Ring 2
x1 = Lx/2
y1 = Ly/2
z1 = Lz

# Resolution
resx         = 16.      #64               # nb of cells in one laser wavelength
resy         = 16.                     
resz         = 16.

# Domain
dx           = l0/resx
dy           = l0/resy
dz           = l0/resz

Nx = Lx/dx
Ny = Ly/dy
Nz = Lz/dz

print(f"Lx: {int(Lx)}")
print(f"Nx: {int(Nx)}")

x = np.linspace(0, int(Lx), int(Nx))
y = np.linspace(0, int(Ly), int(Ny))
z = np.linspace(0, int(Lz), int(Nz))

print("Calculating B components.")
Ax = []
Ay = []
Az = []
num = 0
x_index = 0

dh = display(" ",display_id=True)

for i in x:
    Dx = []
    Dy = []
    Dz = []
    num = num + 1
    dh.update(f"{num} out of {len(x)}")
    for j in y:
        Cx = []
        Cy = []
        Cz = []
        for k in z:
            Cx.append(client.submit(two_rings_x, i,j,k, x0,y0,z0, x1,y1,z1, B0, R))
            Cy.append(client.submit(two_rings_y, i,j,k, x0,y0,z0, x1,y1,z1, B0, R))
            Cz.append(client.submit(two_rings_z, i,j,k, x0,y0,z0, x1,y1,z1, B0, R)) 
        Dx.append(Cx)
        Dy.append(Cy)
        Dz.append(Cz)
    Ax.append(Dx)
    Ay.append(Dy)
    Az.append(Dz)



Lx: 12
Nx: 32
Calculating B components.


'32 out of 32'

In [4]:
def get_result(v):
    return v.result()

print("Calculating B components.")
Bx = []
num = 0
x_index = 0

dh = display(" ",display_id=True)

for i in Ax:
    Dx = []
    num = num + 1
    dh.update(f"{num} out of {len(x)}")
    for j in i:
        Cx = []
        for k in j:
            Cx.append(k.result())
        Dx.append(Cx)
    Bx.append(Dx)

By = []
num = 0
x_index = 0
for i in Ay:
    Dy = []
    num = num + 1
    dh.update(f"{num} out of {len(x)}")
    for j in i:
        Cy = []
        for k in j:
            Cy.append(k.result())
        Dy.append(Cy)
    By.append(Dy)
    
Bz = []
num = 0
x_index = 0
for i in Az:
    Dz = []
    num = num + 1
    dh.update(f"{num} out of {len(x)}")
    for j in i:
        Cz = []
        for k in j:
            Cz.append(k.result())
        Dz.append(Cz)
    Bz.append(Dz)

Calculating B components.


'32 out of 32'

In [6]:
Bx[3][15][8]

-0.04195782639299517

In [7]:
Bx = np.array(Bx)
By = np.array(By)
Bz = np.array(Bz)

In [8]:
Bx[7][3][2]

-0.005053724093361131